In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!rm -rf ./sample_data/

## Get source code
!git clone https://github.com/ivan-fan-dk/DL-final-project.git
!mv DL-final-project/* ./
!rm -rf DL-final-project
!echo "Finished cloning source code."

# Install requirements, will ask you to restart seesion
!pip3 install -q -r requirements.txt

In [ ]:
Q1 = input("What's your model? \n(1) 3GB \n(2) 3GB w/ SEBottleNeck \n(3) 3GB w/ SEBottleNeck w/ augmentation \n(4) 94MB \n(5) 94MB w/ SE\n(6) 3GB w/ augmentation \n")
if Q1 == "1":
    suffix = "3GB"
    datadownloader = "sample_raw_data_downloader.sh"
    datafolder = "DL_Sample_Data"
    trainfile = "train.py"
    testfile = "test.py"
elif Q1 == "2":
    suffix = "3GB_SE"
    datadownloader = "sample_raw_data_downloader.sh"
    datafolder = "DL_Sample_Data"
    trainfile = "se_train.py"
    testfile = "test_se.py"
elif Q1 == "3":
    suffix = "3GB_SE_Aug"
    datadownloader = "sample_raw_data_downloader.sh"
    datafolder = "DL_Sample_Data"
    trainfile = "se_aug_train.py"
    testfile = "test_se.py"
elif Q1 == "4":
    suffix = "94MB"
    datadownloader = "mini_raw_data_downloader.sh"
    datafolder = "DL_Mini_Data"
    trainfile = "train.py"
    testfile = "test.py"
elif Q1 == "5":
    suffix = "94MB_SE"
    datadownloader = "mini_raw_data_downloader.sh"
    datafolder = "DL_Mini_Data"
    trainfile = "se_train.py"
    testfile = "test_se.py"
elif Q1 == "6":
    suffix = "3GB_Aug"
    datadownloader = "sample_raw_data_downloader.sh"
    datafolder = "DL_Sample_Data"
    trainfile = "aug_train.py"
    testfile = "test.py"
else:
    ValueError("Invalid input")

print(f"Selected model: {suffix}")

In [ ]:
# Installing... (Run either "Installing..." or "Loading...". Run this one only if you don't have the data in Google Drive, else run "Loading...")
prompt = input("Install data to Google Drive (1) or just import from Google Drive (2)? ")
if prompt == "1":
    ## Make dir in Google Drive
    !rm -rf /content/gdrive/MyDrive/{datafolder}/
    !mkdir /content/gdrive/MyDrive/{datafolder}/

    ## Start downloading
    !chmod +x ./*.sh
    !./{datadownloader}    # either !./sample_raw_data_downloader.sh or !./raw_data_downloader.sh
elif prompt != "2":
    ValueError("Invalid input")

# Copy data from Google Drive to Google Colab
!echo "Copying data from Google Drive to Google Colab..."
!rm -rf ./{datafolder}/
!cp -r /content/gdrive/MyDrive/{datafolder}/ .
!echo "Finished copying data from Google Drive to Google Colab."

# Datasize
!du -sh ./{datafolder}

In [ ]:
# Train model (LONG TIME)
## data preparation
!rm -rf ./formatted/
!mkdir formatted
!python3 data/prepare_train_data.py ./{datafolder}/ --dataset-format 'kitti_raw' --dump-root ./formatted/data/ --width 416 --height 128 --num-threads 4

## train
!python3 {trainfile} ./formatted/data/ -b4 -m0.2 -s0.1 --epochs 200 --sequence-length 3 --log-output

# Save checkpoints to your Google Drive
!zip -r checkpoints_{suffix}.zip ./checkpoints/
!mv checkpoints_{suffix}.zip gdrive/MyDrive/
!echo "Saved checkpoints_{suffix}.zip in Google Drive."

In [ ]:
# (Optional) Import checkpoints from Google Drive
!cp gdrive/MyDrive/checkpoints_{suffix}.zip .
!unzip -o checkpoints_{suffix}.zip

In [ ]:
# You can change the name of the checkpoints folder here.
selected_checkpoints = f"checkpoints"
%reload_ext tensorboard
%tensorboard --logdir={selected_checkpoints}

In [ ]:
import glob
!rm -rf ./inference_{suffix}/
!rm -rf ./hist_{suffix}/

# If this gives "IndexError: list index out of range" then the following code cannot find the file "dispnet_model_best.pth.tar". Type it manually.
dispnet_model_best_path = glob.glob(f"{selected_checkpoints}/**/dispnet_model_best.pth.tar", recursive=True)[0]
!python3 {testfile} --pretrained {dispnet_model_best_path} --dataset-dir ./test_images/ --output-dir ./inference_{suffix}/ --output-hist-dir ./hist_{suffix}/ --output-depth

In [ ]:
# Save results as zip files in Google Drive
!cp -r ./hist_{suffix}/ gdrive/MyDrive/
!echo "Saved hist_{suffix} in Google Drive."
!cp -r ./inference_{suffix}/ gdrive/MyDrive/
!echo "Saved inference_{suffix} in Google Drive."